In [ ]:
#Libraries
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import os
import pandas as pd
from skimage.feature import hog
import pyshearlab as sl
import pywt
#Machine Learning 
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score

## Descritores
    - Forma
        - SIFT
        - HOG
        - GIST
    - Textura
        - GLCM
        - Haralick
        - LBP

In [ ]:
def roc(X_train, y_train, X_test, y_test, mean_fpr, aucs, tprs, cls, l):
    i = 0
    for c in cls:
        print(c)
        probas_ = c.fit(X_train, y_train).predict_proba(X_test)
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(label_binarize(y_test, classes=[0, 1]), probas_[:, 1])
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC %s (AUC = %0.2f)' % (l[i], roc_auc))
        i += 1
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Luck', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    return(plt)

In [ ]:
classifiers = [
    SVC(kernel="linear", C=0.025, probability=True),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
]
    
cls = ["SVM linear",
       "MLP",
       "AdaBoost",
]

In [ ]:
dir_estomato = "../../data/dataset/STOMA/"
dir_erro = "../../data/dataset/ERRO/"

list_stoma = sorted(os.listdir(dir_estomato))
list_erro = sorted(os.listdir(dir_erro))
img_estomatos = []
img_erros = []



In [ ]:
def dataset(lista):
    '''
    Read the images
    '''
    data1 = []
    data2 = []
    data3 = []
    data4 = []
    data5 = []
    for i in range(0,len(lista),5):
        data1.append(lista[i])
        data2.append(lista[i+1])
        data3.append(lista[i+2])
        data4.append(lista[i+3])
        data5.append(lista[i+4])

    return [data1,data2,data3,data4,data5]

In [ ]:
for i in list_stoma:
    img_estomatos.append(cv2.imread(dir_estomato+i))
for i in list_erro:
    img_erros.append(cv2.imread(dir_erro+i))


In [ ]:
X = np.double(img_estomatos[0])
coeffs = pywt.dwt2(X, 'haar')
cA, (cH, cV, cD) = coeffs
print(cA)

In [ ]:
X = np.double(img_estomatos[0])
useGPU = 1


In [ ]:
X.shape

In [ ]:
shearletSystem = sl.SLgetShearletSystem2D(useGPU,X.shape[0],X.shape[1],4)
coeffs = sl.SLsheardec2D(X,shearletSystem)
print(coeffs)
Xrec = sl.SLshearrec2D(coeffs,shearletSystem)

## HOG

In [ ]:
from skimage.feature import hog
fds_treino_estomato = []
fds_treino_erro = []


for i in img_estomatos:
    fd = hog(i,visualize=False, multichannel=True)
    aux = {k:fd[k] for k in range(len(fd))}
    aux['class'] = 1
    fds_treino_estomato.append(aux)

    
for i in img_erros:
    fd = hog(i,visualize=False, multichannel=True)
    aux = {k:fd[k] for k in range(len(fd))}
    aux['class'] = 0
    fds_treino_erro.append(aux)





In [ ]:
estomato = dataset(fds_treino_estomato)
erro = dataset(fds_treino_erro)



In [ ]:
del fds_treino_estomato
del fds_treino_erro

train1 = pd.DataFrame(sum(estomato[0:4],[])+sum(erro[0:4],[]))
teste1 = pd.DataFrame(estomato[4]+erro[4])


tprs_dall = []
aucs_dall = []
mean_fpr_dall = np.linspace(0, 1, 100)
fig = plt.figure(figsize=(15,15))
imgshow = roc(train1.iloc[:,:-1],train1['class'],teste1.iloc[:,:-1],teste1['class'], mean_fpr_dall, aucs_dall,tprs_dall,classifiers,cls)

del train1
del teste1




In [ ]:
train2 = pd.DataFrame(sum([estomato[0],estomato[1],estomato[2],estomato[4]],[])+
                      sum([erro[0],erro[1],erro[2],erro[4]],[]))
teste2 = pd.DataFrame(estomato[3]+erro[3])


tprs_dall = []
aucs_dall = []
mean_fpr_dall = np.linspace(0, 1, 100)
fig = plt.figure(figsize=(15,15))
imgshow = roc(train2.iloc[:,:-1],train2['class'],teste2.iloc[:,:-1],teste2['class'], mean_fpr_dall, aucs_dall,tprs_dall,classifiers,cls)

del train2
del teste2

In [ ]:

train3 = pd.DataFrame(sum([estomato[0],estomato[1],estomato[3],estomato[4]],[])+
                      sum([erro[0],erro[1],erro[3],erro[4]],[]))
teste3 = pd.DataFrame(estomato[2]+erro[2])


tprs_dall = []
aucs_dall = []
mean_fpr_dall = np.linspace(0, 1, 100)
fig = plt.figure(figsize=(15,15))
imgshow = roc(train3.iloc[:,:-1],train3['class'],teste3.iloc[:,:-1],teste3['class'], mean_fpr_dall, aucs_dall,tprs_dall,classifiers,cls)

del train3
del teste3



In [ ]:
train4 = pd.DataFrame(sum([estomato[0],estomato[2],estomato[3],estomato[4]],[])+
                      sum([erro[0],erro[2],erro[3],erro[4]],[]))
teste4 = pd.DataFrame(estomato[1]+erro[1])

tprs_dall = []
aucs_dall = []
mean_fpr_dall = np.linspace(0, 1, 100)
fig = plt.figure(figsize=(15,15))
imgshow = roc(train4.iloc[:,:-1],train4['class'],teste4.iloc[:,:-1],teste4['class'], mean_fpr_dall, aucs_dall,tprs_dall,classifiers,cls)

del train4
del teste4


In [ ]:

train5 = pd.DataFrame(sum([estomato[3],estomato[1],estomato[2],estomato[4]],[])+
                      sum([erro[3],erro[1],erro[2],erro[4]],[]))
teste5 = pd.DataFrame(estomato[0]+erro[0])

tprs_dall = []
aucs_dall = []
mean_fpr_dall = np.linspace(0, 1, 100)
fig = plt.figure(figsize=(15,15))
imgshow = roc(train5.iloc[:,:-1],train5['class'],teste5.iloc[:,:-1],teste5['class'], mean_fpr_dall, aucs_dall,tprs_dall,classifiers,cls)

del train5
del teste5